In [17]:
%pip install google-cloud-vision

  Using cached google_cloud_vision-3.10.2-py3-none-any.whl (527 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.5/427.5 kB 2.8 MB/s eta 0:00:0000:0100:01
  Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
  Using cached google_api_core-2.26.0-py3-none-any.whl (162 kB)
  Using cached google_auth-2.41.1-py2.py3-none-any.whl (221 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl (294 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 6.7 MB/s eta 0:00:0000:0100:01
  Using cached grpcio_status-1.75.1-py3-none-any.whl (14 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl (11 kB)
  Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl (83 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. 인스타 API로 해시태그 결과 추출

In [21]:
import requests
import json
from datetime import datetime
import os
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import List, Optional

# ✅ .env 로드
load_dotenv()
BASE_URL = os.getenv("API_BASE_URL")

# ✅ DTO 정의
@dataclass
class InstagramPostDTO:
    id: str
    caption: Optional[str]
    media_type: str
    permalink: str
    timestamp: str
    media_urls: List[str]


class InstagramAPI:
    def __init__(self, access_token: str, user_id: str, base_url: str):
        self.access_token = access_token
        self.user_id = user_id
        self.base_url = base_url

    def get_hashtag_id(self, hashtag: str) -> Optional[str]:
        """
        주어진 해시태그 텍스트로부터 Instagram Graph API의 해시태그 ID를 조회
        """
        url = f"{self.base_url}/ig_hashtag_search"
        params = {
            "user_id": self.user_id,
            "q": hashtag,
            "access_token": self.access_token
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json().get("data", [])

            if not data:
                print(f"⚠️ 해시태그 ID를 찾을 수 없습니다: '{hashtag}'")
                return None

            return data[0]["id"]
        except requests.RequestException as e:
            print(f"❌ 해시태그 ID 요청 실패: {e}")
            return None

    def get_recent_media(self, hashtag_id: str, limit: int = 5) -> List[InstagramPostDTO]:
        """
        특정 해시태그 ID에 대한 최근 게시물들을 조회 → DTO 변환
        """
        url = f"{self.base_url}/{hashtag_id}/recent_media"
        params = {
            "user_id": self.user_id,
            "access_token": self.access_token,
            "fields": (
                "id,caption,media_type,media_url,permalink,comments_count,"
                "like_count,timestamp,children{media_url,media_type}"
            ),
            "limit": limit
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            json_data = response.json()
            raw_posts = json_data.get("data", [])

            parsed_posts: List[InstagramPostDTO] = []
            for item in raw_posts:
                if item["media_type"] not in ("IMAGE", "CAROUSEL_ALBUM"):
                    continue

                media_urls: List[str] = []

                if item["media_type"] == "CAROUSEL_ALBUM" and "children" in item:
                    for child in item["children"]["data"]:
                        if "media_url" in child:
                            media_urls.append(child["media_url"])
                elif "media_url" in item:
                    media_urls.append(item["media_url"])

                post_dto = InstagramPostDTO(
                    id=item.get("id"),
                    caption=item.get("caption"),
                    media_type=item.get("media_type"),
                    permalink=item.get("permalink"),
                    timestamp=item.get("timestamp"),
                    media_urls=media_urls
                )
                parsed_posts.append(post_dto)

            return parsed_posts

        except requests.RequestException as e:
            print(f"❌ 요청 실패: {e}")
        except ValueError:
            print("❌ JSON 파싱 실패")
        return []

    def save_json(self, data: List[InstagramPostDTO], filename_prefix="media", hashtag=None):
        """
        DTO 리스트를 JSON 파일로 저장
        """
        # hashtag_part = f"_{hashtag}" if hashtag else ""
        # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        # filename = f"{filename_prefix}{hashtag_part}_{timestamp}.json"
        filename = "popup.json"  # ✅ 파일 이름 고정

        # dataclass → dict 변환
        json_data = [post.__dict__ for post in data]

        try:
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(json_data, f, indent=4, ensure_ascii=False)
            print(f"📁 저장 완료: {os.path.abspath(filename)}")
        except Exception as e:
            print(f"❌ 저장 실패: {e}")

    @staticmethod
    def play():
        """
        실행 진입점
        """
        load_dotenv()
        access_token = os.getenv("INSTA_ACCESS_TOKEN")
        user_id = os.getenv("IG_USER_ID")
        hashtag = "팝업스토어"
        base_url = "https://graph.facebook.com/v20.0"

        if not access_token or not user_id:
            print("❌ .env 설정 누락: ACCESS_TOKEN 또는 IG_USER_ID")
            return

        api = InstagramAPI(access_token, user_id, base_url)
        hashtag_id = api.get_hashtag_id(hashtag)
        if not hashtag_id:
            print(f"❌ 해시태그 ID를 찾을 수 없습니다: {hashtag}")
            return

        posts = api.get_recent_media(hashtag_id)
        api.save_json(posts, hashtag=hashtag)


if __name__ == "__main__":
    InstagramAPI.play()


📁 저장 완료: /Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/popup.json


## 2. GPT API로 데이터 정제

In [38]:
import os
import re
import json
import time
import requests
from dotenv import load_dotenv
from datetime import datetime
from urllib.parse import urlparse
from dataclasses import dataclass
from typing import List, Optional


# ==============================
# 📦 DTO 정의
# ==============================

@dataclass
class InstagramPostDTO:
    """📸 Instagram 원본 데이터"""
    id: str
    caption: str
    media_type: str
    permalink: str
    media_urls: List[str]

@dataclass
class GptParsedEventDTO:
    """🧠 GPT 파싱 결과"""
    name: str
    start_date: str
    end_date: str
    open_time: str
    close_time: str
    address: str
    region: str
    geocoding_query: str
    caption_summary: str
    section: Optional[int] = None

@dataclass
class PopupEventDTO:
    """📌 최종 병합 결과"""
    name: str
    start_date: str
    end_date: str
    open_time: str
    close_time: str
    address: str
    region: str
    geocoding_query: str
    insta_post_id: str
    insta_post_url: str
    caption_summary: str
    caption: str
    image_url: List[str]
    image_paths: List[str]
    media_type: str


# ==============================
# 🧠 GPT 파이프라인
# ==============================

class GptAPI:
    REQUIRED_FIELDS = ["name", "start_date", "end_date", "address", "region", "caption_summary"]  # ✅ 추가
    def __init__(self, access_token, model="gpt-4o-mini"):
        self.access_token = access_token
        self.model = model
        self.endpoint = "https://api.openai.com/v1/chat/completions"
        self.session = requests.Session()
        self.session.headers.update({
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json",
        })

    # ---------- 문자열 정제 ----------
    @staticmethod
    def slugify(text: str) -> str:
        if not text:
            return "no_name"
        text = text.strip()
        text = re.sub(r'\s+', '_', text)
        text = re.sub(r'[^\w\-가-힣]', '', text)
        return text

    # ---------- 파일 입출력 ----------
    def file_open(self, filename: str) -> List[InstagramPostDTO]:
        with open(filename, "r", encoding="utf-8") as f:
            raw_data = json.load(f)

        posts: List[InstagramPostDTO] = []
        for item in raw_data:
            posts.append(
                InstagramPostDTO(
                    id=item.get("id", ""),
                    caption=item.get("caption", ""),
                    media_type=item.get("media_type", ""),
                    permalink=item.get("permalink", ""),
                    media_urls=item.get("media_urls", []),
                )
            )
        return posts

    def file_save(self, data: List[PopupEventDTO]):
        path = "gpt.json"
        json_data = [obj.__dict__ for obj in data]
        with open(path, "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        print(f"📁 저장 완료: {os.path.abspath(path)}")
        return os.path.abspath(path)

    # ---------- GPT 프롬프트 (원문 유지) ----------
    def build_prompt(self, sections):
        lines = []
        for idx, cap in sections:
            lines.append(f"[section {idx}]\n{cap}")
        body = "\n\n---\n\n".join(lines)

        # ✅ 여기서 required_fields 문자열화
        required_list_str = ", ".join(self.REQUIRED_FIELDS)
        return f"""
        아래에는 여러 개의 '섹션' 텍스트가 주어집니다.
        각 섹션에는 하나 이상의 팝업 이벤트 정보가 포함될 수 있습니다.

        각 팝업 이벤트에 대해 다음 **소문자 키**만 포함된 객체를 생성하고,
        모든 객체를 **JSON 배열**로 반환하세요.

        ─────────────────────────────
        📌 필드별 작성 규칙
        ─────────────────────────────

        - name: 팝업 이름 또는 행사명

        - start_date: 시작 날짜 (YYYY-MM-DD 형식)

        - end_date: 종료 날짜 (YYYY-MM-DD 형식)

        - open_time: 운영 시작 시간 (HH:MM)

        - close_time: 운영 종료 시간 (HH:MM)

        - address: 도로명 주소 또는 건물명
            - ⚠️ address가 추출되지 않는 경우, 이 이벤트는 JSON에 포함하지 마세요.

        - region: 지역명 (예: 서울, 부산, 도쿄)
            - ⚠️ region이 누락되면 이 이벤트는 JSON에 포함하지 마세요.

        - geocoding_query: address와 region, name을 기반으로 지오코딩 API 검색에 최적화된 문자열
            1) 지역(region)을 반드시 가장 앞에 붙이세요. (예: '부산', '서울', '성남')
            2) address 또는 name에서 브랜드명, 건물명, 공간명 등의 핵심 지명 요소만 붙이세요.
                - 예: "성남 현대백화점 판교" ✅
                - 예: "성남 현대백화점 판교 도씨" ❌ (브랜드명 제거)
            3) 층수, 방향, 조사 등 불필요한 단어는 제거하세요:
                - 'B1', '1층', '2F', 'B2F', '지하 1층'
                - '앞', '근처', '맞은편', '옆', '뒷편', '앞쪽', '뒤편'
                - '~에서', '~앞', '~근처', '~맞은편'
            4) 브랜드명, 팝업 이름 등 검색 정확도를 떨어뜨리는 불필요한 단어는 포함하지 마세요.
            5) 문장형이 아니라 짧고 검색 최적화된 명사구로 반환하세요.
            6) 단순히 지역명만 쓰는 것은 ❌ 금지입니다. (예: "성남" ❌)
            → 최소한 지역명 + 지명/건물명까지 포함해야 합니다. (예: "성남 현대백화점 판교" ✅)
            7) address가 비어 있는 경우에는 name에서 지명 요소만 추출해 사용하세요. 브랜드명은 포함하지 마세요.

        - section: 이 이벤트가 추출된 섹션 번호(정수)

        - caption_summary: "caption_summary는 단순 요약이 아니라, 인스타 원문을 기반으로 한 완성된 게시글형 문단입니다. \
            ✨ 구성 규칙:\n
            - 전체는 약 6~10줄로 구성하세요.\n
            - 상단에는 팝업 이름, 위치, 일정, 운영시간을 간결히 표시하세요.\n
            - 하단에는 팝업의 분위기, 전시·체험 내용, 운영 특징 등을 자연스럽게 3줄 이상으로 설명하세요.\n
            - 문장은 짧고 자연스럽게, 말하듯이 표현하세요.\n
            - 감성적인 표현은 허용하지만, 과장되거나 홍보성 어투는 피하세요.\n
            - 문장 사이에는 줄바꿈(\n)을 포함하세요.\n
            예시:\n
            🥐 Jam in Bread 팝업스토어 오픈\n
            📍 신세계백화점 강남점 B1\n
            📅 10.7(월) ~ 10.13(일)\n
            🕥 10:30AM ~ 8:00PM\n
            \n
            따뜻한 향이 퍼지는 공간에서 잼과 빵을 함께 즐길 수 있습니다.\n
            다양한 수제잼과 베이커리 굿즈가 전시되어 있고, 일부 상품은 현장 구매도 가능합니다.\n
            하루의 시작을 부드럽게 채워주는 작은 휴식 같은 팝업입니다.\n"

        ─────────────────────────────
        ❗ 포함 기준
        ─────────────────────────────
        - 반드시 다음 필드들이 모두 존재해야 합니다:
        {required_list_str}
        - 위 필드 중 하나라도 누락된 이벤트는 JSON에 포함하지 마세요.
        → 불명확하거나 주소/날짜가 없는 이벤트는 제외하세요.

        ─────────────────────────────
        📅 날짜 규칙
        ─────────────────────────────
        - '10/7~10/23'처럼 월/일만 있으면 2025년으로 보완하세요.
        - 과거년도(2023, 2024 등)가 명시되어 있으면 그대로 사용하세요.
        - 모호한 날짜는 제외하세요.

        ─────────────────────────────
        🕒 시간 규칙
        ─────────────────────────────
        - '10:00~20:00' 형태는 open_time / close_time으로 나누세요.
        - 명시 없으면 빈 문자열로 둡니다.

        ─────────────────────────────
        🖼 이미지 규칙
        ─────────────────────────────
        - 이미지가 여러 장이면 배열 형태로 반환하세요.
        - 단일 이미지도 배열로 감싸서 반환하세요.

        ─────────────────────────────
        ⚠️ 출력 형식
        ─────────────────────────────
        - 반드시 JSON 배열([])만. 설명/주석/코드블록 금지.

        {body}
        """


    # ---------- GPT 호출 ----------
    def call_gpt(self, prompt, max_tokens=1500, retries=2):
        payload = {
            "model": self.model,
            "temperature": 0,
            "messages": [
                {"role": "system", "content": "너는 텍스트에서 구조화된 정보를 추출하는 전문가야."},
                {"role": "user", "content": prompt},
            ],
            "max_tokens": max_tokens
        }
        for attempt in range(retries + 1):
            try:
                resp = self.session.post(self.endpoint, json=payload, timeout=60)
                if resp.status_code == 200:
                    return resp.json()["choices"][0]["message"]["content"]
                else:
                    print(f"⚠️ 응답 오류: {resp.status_code}")
                time.sleep(1)
            except requests.RequestException as e:
                print(f"⚠️ 요청 실패: {e}")
                time.sleep(1)
        raise RuntimeError("❌ GPT 응답 실패")

    # ---------- GPT 결과 파싱 ----------
    def extract_json_array(self, text) -> List[GptParsedEventDTO]:
        code_match = re.search(r"```(?:json)?\s*(\[[\s\S]*?\])\s*```", text)
        candidate = code_match.group(1) if code_match else self._greedy_bracket_slice(text)
        try:
            data = json.loads(candidate)
            return self._normalize_schema(data)
        except json.JSONDecodeError:
            return []

    def _normalize_schema(self, data) -> List[GptParsedEventDTO]:
        if not isinstance(data, list):
            return []
        parsed = []
        for obj in data:
            parsed.append(
                GptParsedEventDTO(
                    name=(obj.get("name") or "").strip(),
                    start_date=(obj.get("start_date") or "").strip(),
                    end_date=(obj.get("end_date") or "").strip(),
                    open_time=(obj.get("open_time") or "").strip(),
                    close_time=(obj.get("close_time") or "").strip(),
                    address=(obj.get("address") or "").strip(),
                    region=(obj.get("region") or "").strip(),
                    geocoding_query=(obj.get("geocoding_query") or "").strip(),
                    caption_summary=(obj.get("caption_summary") or "").strip(),
                    section=int(obj["section"]) if obj.get("section") is not None else None
                )
            )
        return parsed

    def _greedy_bracket_slice(self, text):
        start, end = text.find("["), text.rfind("]")
        return text[start:end + 1] if start != -1 and end != -1 else "[]"
    
    # ---------- 필수 필드 필터 ----------
    def filter_required_fields(self, events: List[PopupEventDTO]) -> List[PopupEventDTO]:
        valid = []
        for e in events:
            missing = [f for f in self.REQUIRED_FIELDS if not getattr(e, f, "").strip()]
            if missing:
                print(f"⚠️ 필수 필드 누락({missing}) → {e.name or '이름 없음'} 제외")
                continue
            valid.append(e)
        return valid

    # ---------- 원본 데이터 병합 ----------
    def _enrich_with_original(self, posts: List[InstagramPostDTO], extracted: List[GptParsedEventDTO], section_to_post):
        results: List[PopupEventDTO] = []
        for event in extracted:
            orig = section_to_post.get(event.section, InstagramPostDTO("", "", "", "", []))
            results.append(
                PopupEventDTO(
                    name=event.name,
                    start_date=event.start_date,
                    end_date=event.end_date,
                    open_time=event.open_time,
                    close_time=event.close_time,
                    address=event.address,
                    region=event.region,
                    geocoding_query=event.geocoding_query,
                    insta_post_id=orig.id,
                    insta_post_url=orig.permalink,
                    caption_summary=event.caption_summary,
                    caption=orig.caption,
                    image_url=orig.media_urls,
                    image_paths=[],  # 다운로드 후 채워짐
                    media_type=orig.media_type,
                )
            )
        return results

    # ---------- 이미지 다운로드 ----------
    def download_images(self, event: PopupEventDTO, base_dir="images"):
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        name_slug = self.slugify(event.name) or "no_name"
        folder_name = f"{timestamp}_{event.insta_post_id}"
        folder_path = os.path.join(base_dir, folder_name)

        try:
            os.makedirs(folder_path, exist_ok=True)
        except Exception as e:
            print(f"❌ 폴더 생성 실패 ({folder_path}): {e}")
            return

        image_paths = []
        valid_image_urls = []  # ✅ webp 제거 후 다시 담을 리스트

        for idx, url in enumerate(event.image_url, start=1):
            try:
                if not url or not url.startswith("http"):
                    print(f"⚠️ 잘못된 URL → 스킵: {url}")
                    continue

                parsed = urlparse(url)
                ext = os.path.splitext(parsed.path)[1].lower().split("?")[0]

                # 🚫 webp 제외 — 아예 URL 리스트에서도 제거
                if ext == ".webp":
                    print(f"🚫 webp 파일 스킵 및 URL 제거: {url}")
                    continue

                if ext in ("", ".heic"):
                    ext = ".jpg"

                filename = f"{name_slug}_{idx}{ext}"
                filepath = os.path.join(folder_path, filename)

                resp = requests.get(url, timeout=20)
                if resp.status_code != 200:
                    print(f"⚠️ 다운로드 실패 (status={resp.status_code}): {url}")
                    continue

                content_type = resp.headers.get("Content-Type", "").lower()
                if not content_type.startswith("image/"):
                    print(f"⚠️ 이미지 아님 (Content-Type={content_type}): {url}")
                    continue

                with open(filepath, "wb") as f:
                    f.write(resp.content)

                image_paths.append(os.path.abspath(filepath))
                valid_image_urls.append(url)  # ✅ 실제로 성공한 URL만 유지
                print(f"✅ 이미지 저장 완료: {filepath}")

            except Exception as e:
                print(f"❌ 이미지 다운로드 처리 중 오류 ({url}): {e}")

        event.image_paths = image_paths
        event.image_url = valid_image_urls  # ✅ webp 제거 반영



    # ---------- 전체 파이프라인 ----------
    def process_file(self, filename, batch_size=10, download=False):
        posts = self.file_open(filename)
        sections = []
        section_to_post = {}
        for idx, post in enumerate(posts):
            if post.caption:
                sections.append((idx, post.caption))
                section_to_post[idx] = post

        if not sections:
            print("⚠️ 캡션 텍스트가 없습니다.")
            return []

        all_extracted: List[GptParsedEventDTO] = []
        for i in range(0, len(sections), batch_size):
            chunk = sections[i:i + batch_size]
            prompt = self.build_prompt(chunk)
            try:
                resp_text = self.call_gpt(prompt)
                extracted = self.extract_json_array(resp_text)
                all_extracted.extend(extracted)
            except Exception as e:
                print(f"⚠️ GPT 처리 실패: {e}")

        results = self._enrich_with_original(posts, all_extracted, section_to_post)

        if download:
            for event in results:
                self.download_images(event)

        # ✅ 필수 필드 필터링 추가
        before_len = len(results)
        results = self.filter_required_fields(results)
        after_len = len(results)
        print(f"📌 필수 필드 필터링: {before_len - after_len}건 제외됨")

        # 🧹 이미지 없는 팝업 제거
        before_len = len(results)
        results = [event for event in results if len(event.image_url) > 0 or len(event.image_paths) > 0]
        after_len = len(results)

        print(f"🧾 총 {before_len}건 중 {before_len - after_len}건은 이미지 없음으로 제외됨")

        return results


    # ---------- 실행 ----------
    @staticmethod
    def play(download=False):
        load_dotenv()
        token = os.getenv("GPT_ACCESS_TOKEN")
        if not token:
            print("❌ 환경 변수 누락: GPT_ACCESS_TOKEN")
            return
        api = GptAPI(token)
        results = api.process_file("popup.json", batch_size=10, download=download)
        api.file_save(results)


# ==============================
# 🏁 실행
# ==============================
if __name__ == "__main__":
    GptAPI.play(download=True)


🚫 webp 파일 스킵 및 URL 제거: https://scontent-icn2-1.cdninstagram.com/v/t51.82787-15/567632338_18340038574206293_4434618611890754533_n.webp?stp=dst-jpg_e35_tt6&_nc_cat=104&ccb=1-7&_nc_sid=18de74&efg=eyJlZmdfdGFnIjoiQ0FST1VTRUxfSVRFTS5iZXN0X2ltYWdlX3VybGdlbi5DMyJ9&_nc_ohc=gaZQQLuJehMQ7kNvwGs6frV&_nc_oc=AdktRynIXALn0P4dY5ctePqWCixkDnx4c0FDO8X-TonnviManNFfiMA60gxRRjyS5xA&_nc_zt=23&_nc_ht=scontent-icn2-1.cdninstagram.com&edm=AEoDcc0EAAAA&_nc_gid=c_hVnWAHUTEMR97esJFHEw&oh=00_AfcdupKn6WXzi0pzzNgbjjz09nPuAuMJxha0XpBw88u_Zg&oe=68FBB42A
🚫 webp 파일 스킵 및 URL 제거: https://scontent-icn2-1.cdninstagram.com/v/t51.82787-15/568704543_18340038583206293_3042410553993053055_n.webp?stp=dst-jpg_e35_tt6&_nc_cat=102&ccb=1-7&_nc_sid=18de74&efg=eyJlZmdfdGFnIjoiQ0FST1VTRUxfSVRFTS5iZXN0X2ltYWdlX3VybGdlbi5DMyJ9&_nc_ohc=Z3dwpDlblKAQ7kNvwFh49Yt&_nc_oc=AdlKJK-sNedM_fQA4-nY5h60ZlAlPulq_YJcMEgkIevz6gY6JH3wi2tuRa4J59EVKxA&_nc_zt=23&_nc_ht=scontent-icn2-1.cdninstagram.com&edm=AEoDcc0EAAAA&_nc_gid=c_hVnWAHUTEMR97esJFHEw&oh=00_Afc

## 3. 도로명주소 및 위경도 추가(둘중 하나라도 없으면 필터링)

In [39]:
# GeoCoding.py
import requests
import urllib.parse
import os
import json
from dataclasses import dataclass
from dotenv import load_dotenv
from typing import Optional, List


# ==============================
# 📦 DTO 정의
# ==============================

@dataclass
class PlaceInfoDTO:
    road_address: Optional[str]
    longitude: Optional[float]
    latitude: Optional[float]


# ==============================
# 🧭 GeoCoding 로직
# ==============================

class GeoCoding:
    """
    GPT 결과 JSON에 도로명주소 및 좌표(경도/위도)를 추가하는 클래스.
    - Naver Local Search API를 사용 (장소명 기반)
    - 결과를 popup_with_geo.json으로 저장
    """

    def __init__(self):
        load_dotenv()
        self.client_id = os.getenv("CLIENT_ID")
        self.client_secret = os.getenv("CLIENT_SECRET")
        if not self.client_id or not self.client_secret:
            raise ValueError("❌ CLIENT_ID / CLIENT_SECRET 환경 변수가 누락되었습니다.")

    # -----------------------------------
    # 📍 1. 장소 검색 (Naver Local API)
    # -----------------------------------
    def get_place_info(self, query: str) -> PlaceInfoDTO:
        """장소명으로 검색 후 주소와 좌표 반환"""
        encoded_query = urllib.parse.quote(query)
        url = f"https://openapi.naver.com/v1/search/local.json?query={encoded_query}&display=1"
        headers = {
            "X-Naver-Client-Id": self.client_id,
            "X-Naver-Client-Secret": self.client_secret
        }

        try:
            response = requests.get(url, headers=headers, timeout=10)
            data = response.json()

            if 'items' in data and data['items']:
                item = data['items'][0]
                road_address = item.get('roadAddress') or item.get('address')

                # 좌표값이 비어있는 경우도 대비
                try:
                    longitude = float(item['mapx']) / 10_000_000 if item.get('mapx') else None
                    latitude = float(item['mapy']) / 10_000_000 if item.get('mapy') else None
                except Exception:
                    longitude, latitude = None, None

                road_address = self.normalize_address(road_address)

                return PlaceInfoDTO(
                    road_address=road_address,
                    longitude=longitude,
                    latitude=latitude
                )

        except Exception as e:
            print(f"⚠️ Geocoding 실패 ({query}): {e}")

        return PlaceInfoDTO(
            road_address=None,
            longitude=None,
            latitude=None
        )

    # -----------------------------------
    # 🪄 주소 변환 로직
    # -----------------------------------
    def normalize_address(self, address: Optional[str]) -> Optional[str]:
        if not address:
            return address

        replacements = {
            "서울특별시": "서울",
            "부산광역시": "부산",
            "대전광역시": "대전",
            "대구광역시": "대구",
            "인천광역시": "인천",
            "광주광역시": "광주",
            "울산광역시": "울산",
            "세종특별자치시": "세종",
            "제주특별자치도": "제주"
        }

        original = address
        for old, new in replacements.items():
            if old in address:
                address = address.replace(old, new)

        parts = address.split(" ", 1)
        if parts and parts[0].endswith("시"):
            parts[0] = parts[0].removesuffix("시")
        address = " ".join(parts)

        if address == original:
            print(f"⚠️ 치환 대상 아님: {original}")

        return address

    # -----------------------------------
    # 🗺 2. popup_refined.json → 좌표추가
    # -----------------------------------
    def add_geocoding_to_json(
        self,
        input_file: str = "gpt.json",
        output_file: str = "geo.json"
    ):
        """
        입력 JSON에 road_address / longitude / latitude 추가 후 저장
        geocoding_query 필드를 우선 사용하고, 없으면 address 사용
        위경도 값이 없을 경우 필터링
        """
        if not os.path.exists(input_file):
            print(f"❌ 입력 파일 없음: {input_file}")
            return

        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        enriched = []
        skipped = 0
        for event in data:
            query = event.get("geocoding_query") or event.get("address")
            if not query:
                print(f"⚠️ 지오코딩 대상 없음: {event.get('name')}")
                skipped += 1
                continue

            place_info = self.get_place_info(query)

            # 📌 위경도 값 없는 경우 제외
            if place_info.longitude is None or place_info.latitude is None:
                print(f"🚫 위경도 없음 → 스킵: {event.get('name')} ({query})")
                skipped += 1
                continue

            # ✅ 정상 데이터만 추가
            new_event = {**event}
            new_event["road_address"] = place_info.road_address
            new_event["longitude"] = place_info.longitude
            new_event["latitude"] = place_info.latitude

            enriched.append(new_event)

        # 3️⃣ 저장
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(enriched, f, ensure_ascii=False, indent=2)

        print(f"✅ Geocoding 완료: {output_file} (총 {len(enriched)}건, 스킵 {skipped}건)")

    # -----------------------------------
    # 🚀 3. 실행 메서드
    # -----------------------------------
    @staticmethod
    def play():
        geo = GeoCoding()
        geo.add_geocoding_to_json()


if __name__ == "__main__":
    GeoCoding.play()

⚠️ 치환 대상 아님: 경기도 성남시 분당구 판교역로146번길 20 현대백화점 판교점
✅ Geocoding 완료: geo.json (총 1건, 스킵 0건)


## mysql

In [40]:
import os
import json
import requests
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import List, Optional
from datetime import datetime

# ✅ Vision 기능 import
import VisionAPI


# ==============================
# 📦 DTO 정의
# ==============================

@dataclass
class PopupImageDTO:
    imageUrl: str
    sortOrder: int

@dataclass
class PopupUploadDTO:
    name: str
    startDate: str
    endDate: str
    openTime: Optional[str]
    closeTime: Optional[str]
    address: str
    roadAddress: Optional[str]
    longitude: Optional[float]
    latitude: Optional[float]
    region: str
    geocodingQuery: str
    instaPostId: str
    instaPostUrl: str
    captionSummary: str
    caption: str
    mediaType: str
    imageUrl: Optional[str]
    imageList: List[PopupImageDTO]
    recommendIds: List[int]
    isActive: bool = True


# ==============================
# 🧭 유틸 함수
# ==============================

def convert_to_public_path(local_path: str) -> str:
    """절대 경로를 /images/... 상대 경로로 변환"""
    if "/images/" in local_path:
        idx = local_path.index("/images/")
        return local_path[idx:]
    return local_path


def build_image_list(image_paths: List[str]) -> List[PopupImageDTO]:
    """image_paths → DTO 리스트 변환"""
    return [
        PopupImageDTO(
            imageUrl=convert_to_public_path(path),
            sortOrder=idx
        )
        for idx, path in enumerate(image_paths)
    ]


def build_payload(item: dict) -> PopupUploadDTO:
    """원본 dict → DTO 변환"""
    image_paths = item.get("image_paths", [])
    image_list = build_image_list(image_paths)
    image_urls = item.get("image_url", [])
    image_url_first = image_urls[0] if image_urls else None

    return PopupUploadDTO(
        name=item.get("name"),
        startDate=item.get("start_date"),
        endDate=item.get("end_date"),
        openTime=item.get("open_time"),
        closeTime=item.get("close_time"),
        address=item.get("address"),
        roadAddress=item.get("road_address"),
        longitude=item.get("longitude"),
        latitude=item.get("latitude"),
        region=item.get("region"),
        geocodingQuery=item.get("geocoding_query"),
        instaPostId=item.get("insta_post_id"),
        instaPostUrl=item.get("insta_post_url"),
        captionSummary=item.get("caption_summary"),
        caption=item.get("caption"),
        mediaType=item.get("media_type"),
        imageUrl=image_url_first,
        imageList=image_list,
        recommendIds=[1],
    )


# ==============================
# 🐬 Mysql 업로드 클래스
# ==============================

class Mysql:
    @staticmethod
    def send_popup(item: dict, api_url: str) -> bool:
        """Vision 통과한 팝업 정보를 API로 업로드"""
        payload_dto = build_payload(item)
        payload = json.loads(json.dumps(payload_dto, default=lambda o: o.__dict__))

        print("\n📤 업로드 요청 payload:")
        print(json.dumps(payload, ensure_ascii=False, indent=2))

        headers = {"Content-Type": "application/json"}
        response = requests.post(api_url, json=payload, headers=headers, timeout=10)

        if response.status_code == 200:
            print(f"✅ 업로드 성공: {payload_dto.instaPostId}")
            return True
        else:
            print(f"❌ 업로드 실패 ({response.status_code}): {payload_dto.instaPostId}")
            print(response.text)
            return False

    @staticmethod
    def play():
        # ✅ 환경 변수 로드
        BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
        env_path = os.path.join(BASE_DIR, ".env")
        file_path = os.path.join(BASE_DIR, "geo.json")
        load_dotenv(dotenv_path=env_path, override=True)

        API_URL = os.getenv("POPUP_API_URL", "https://poppang.co.kr/api/v1/popup")

        print(f"📂 BASE_DIR: {BASE_DIR}")
        print(f"📄 JSON 경로: {file_path} → 존재? {os.path.exists(file_path)}")
        print(f"🌐 API URL: {API_URL}")

        if not os.path.exists(file_path):
            print("❌ popup_with_geo.json 파일이 없습니다.")
            return

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        total = len(data)
        inserted = 0
        skipped = 0
        human_skipped = 0

        for item in data:
            insta_post_id = item.get("insta_post_id")
            media_type = item.get("media_type")
            image_paths = item.get("image_paths", [])

            print(f"\n📌 처리중: insta_post_id={insta_post_id}")
            print(f"📸 이미지 경로: {image_paths}")

            # 🎥 VIDEO Vision 스킵
            if media_type == "VIDEO":
                print(f"🎥 VIDEO 타입 → Vision 검사 스킵")
                if Mysql.send_popup(item, API_URL):
                    inserted += 1
                else:
                    skipped += 1
                continue

            # 🧠 Vision 검사 (IMAGE, CAROUSEL_ALBUM)
            if image_paths:
                try:
                    has_human = VisionAPI.contains_human_in_all_files(image_paths)
                    if has_human:
                        human_skipped += 1
                        print(f"🚫 Vision 감지됨 → 업로드 스킵 insta_post_id={insta_post_id}")
                        continue
                except Exception as e:
                    print(f"❌ Vision 검사 중 오류: {e}")
                    skipped += 1
                    continue
            else:
                print(f"⚠️ image_paths 비어있음 → Vision 검사 스킵")

            # ✅ Vision 통과 후 업로드
            if Mysql.send_popup(item, API_URL):
                inserted += 1
            else:
                skipped += 1

        print("\n===============================")
        print(f"✅ 업로드 완료: {inserted}/{total}")
        print(f"🚫 Vision 필터로 스킵: {human_skipped}")
        print(f"⚠️ 오류 또는 기타 스킵: {skipped}")
        print("===============================")


# ==============================
# 🏁 실행
# ==============================
if __name__ == "__main__":
    Mysql.play()


📂 BASE_DIR: /Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI
📄 JSON 경로: /Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/geo.json → 존재? True
🌐 API URL: https://poppang.co.kr/api/v1/popup

📌 처리중: insta_post_id=18105581458626493
📸 이미지 경로: ['/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_1.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_2.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_3.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_4.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_5.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626493/도씨_팝업스토어_6.jpg', '/Users/kimdonghyeon/2025/개발/SwiftLab/Lab/RestAPI/images/20251020-161558_18105581458626